In [2]:
from nba_api.stats.endpoints import playergamelog, commonallplayers
import pandas as pd
import time
import numpy as np

# Récupérer la liste complète des joueurs (actuels et historiques)
players_data = commonallplayers.CommonAllPlayers(is_only_current_season=0)
players_df = players_data.get_data_frames()[0]
print(f"Nombre total de joueurs dans la base NBA: {len(players_df)}")

# Définir les saisons à analyser (dernières saisons)
seasons = [
    '2015-16', '2016-17', '2017-18', '2018-19', '2019-20',
    '2020-21', '2021-22', '2022-23', '2023-24', '2024-25'
]

# Liste pour stocker toutes les statistiques
all_player_stats = []

# Mécanisme de sauvegarde
save_interval = 20  # Sauvegarde tous les 20 joueurs
save_counter = 0

# Dictionnaire pour suivre les joueurs déjà traités
processed_players = {}

# Récupérer tous les joueurs ayant participé à chaque saison
for season in seasons:
    print(f"=== Traitement de la saison {season} ===")
    
    # On peut utiliser leaguedashplayerstats pour obtenir tous les joueurs d'une saison
    from nba_api.stats.endpoints import leaguedashplayerstats
    
    try:
        # Récupérer les stats basiques de tous les joueurs pour cette saison
        player_stats = leaguedashplayerstats.LeagueDashPlayerStats(season=season)
        season_players = player_stats.get_data_frames()[0]
        
        print(f"Nombre de joueurs pour la saison {season}: {len(season_players)}")
        
        # Pour chaque joueur de cette saison
        for _, player in season_players.iterrows():
            player_id = player['PLAYER_ID']
            player_name = player['PLAYER_NAME']
            
            # Vérifier si on a déjà traité ce joueur pour cette saison
            player_season_key = f"{player_id}_{season}"
            if player_season_key in processed_players:
                continue
            
            processed_players[player_season_key] = True
            
            print(f"Récupération des données pour {player_name} (ID: {player_id}) - Saison {season}...")
            
            try:
                # Récupérer le game log complet du joueur pour cette saison
                gamelog = playergamelog.PlayerGameLog(player_id=player_id, season=season)
                stats = gamelog.get_data_frames()[0]
                
                if not stats.empty:
                    # Ajouter des informations supplémentaires
                    stats['PLAYER_NAME'] = player_name
                    stats['PLAYER_ID'] = player_id
                    stats['SEASON'] = season
                    
                    all_player_stats.append(stats)
                    print(f"  {len(stats)} matchs trouvés")
                else:
                    print(f"  Aucun match trouvé")
                    
                # Pause pour éviter de surcharger l'API
                time.sleep(np.random.uniform(0.1, 0.5))
                
            except Exception as e:
                print(f"  Erreur: {e}")
            
            # Sauvegarde intermédiaire
            save_counter += 1
            if save_counter % save_interval == 0:
                if all_player_stats:
                    temp_df = pd.concat(all_player_stats, ignore_index=True)
                    temp_df.to_csv(f"nba_players_data_temp1_{save_counter}.csv", index=False)
                    print(f"Sauvegarde intermédiaire effectuée ({len(temp_df)} entrées)")
                
    except Exception as e:
        print(f"Erreur lors de la récupération des joueurs pour la saison {season}: {e}")

# Combiner toutes les statistiques des joueurs
if all_player_stats:
    players_stats_df = pd.concat(all_player_stats, ignore_index=True)
    print(f"Total: {len(players_stats_df)} entrées de matchs de joueurs")
    print(f"Nombre total de joueurs uniques: {players_stats_df['PLAYER_ID'].nunique()}")
    
    # Enregistrer les données finales
    players_stats_df.to_csv("nba_players_historical_data.csv", index=False)
    print("Données sauvegardées dans nba_players_historical_data.csv")
else:
    print("Aucune donnée de joueur récupérée")

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)